In [1]:
from google.colab import drive
import os
if 'google.colab' in str(get_ipython()):
  drive.mount('/content/gdrive')
  os.chdir('/content/gdrive/MyDrive/Benchmarking-Distributional-Alignment-of-Small-Language-Models')

Mounted at /content/gdrive


In [2]:
import os
from google.colab import userdata
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [3]:
!pip install --upgrade -q transformers huggingface_hub peft \
  accelerate bitsandbytes datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 re

In [4]:
from huggingface_hub import login
import tensorflow as tf
login(os.environ["HF_TOKEN"])
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
# Run random baseline to get these files first!
n_training_qIDs = "train_qIDs.json"
n_testing_qIDs = "test_qIDs.json"

# demographic group and output type
demographic_group = "POLPARTY"
demographic = "Republican"
output_type = "sequence"

In [6]:
model_id = "google/gemma-2-2b-it"
device = "cuda"

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Let's quantize the model to reduce its weight
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Let's load the final model
model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model.gradient_checkpointing_enable()

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [8]:
from utils import *
import json

sampled_qIDs = json.load(open(n_training_qIDs))

qID_datasets = []
for qID in sampled_qIDs:
    qID_dataset = get_few_shot_training_examples(
        qID,
        wave="Pew_American_Trends_Panel_disagreement_100",
        demographic_group=demographic_group,
        demographic=demographic,
        output_type=output_type,
        dataset="opinionqa",
        n_shots=5,
        n_simulations_per_shot=1,
    )
    qID_datasets += [qID_dataset]
raw_dataset = pd.concat(qID_datasets)
training_dataset = prepare_df(raw_dataset.copy(), tokenizer).reset_index(drop=True)
training_dataset.head(3)

/content/gdrive/MyDrive/Benchmarking-Distributional-Alignment-of-Small-Language-Models/utils.py:138: RuntimeWarning: invalid value encountered in divide
  icl_values = np.array(icl_values)/np.sum(icl_values)


,input,output,qID,icl_qID,demographic_group,demographic,output_type,wave
0,<start_of_turn>user\nPlease simulate 30 sample...,Answer: D D D B C B C D C D D D A C D B D B D ...,SOCIETY_SSM_W92,GAYMARR2_W32,POLPARTY,Republican,sequence,Pew_American_Trends_Panel_disagreement_100
1,<start_of_turn>user\nPlease simulate 30 sample...,Answer: C C E D B B C B E C E D C B B B B E C ...,SOCIETY_SSM_W92,FAMSURV6_W50,POLPARTY,Republican,sequence,Pew_American_Trends_Panel_disagreement_100
2,<start_of_turn>user\nPlease simulate 30 sample...,Answer: F A E C C C C C F A F E D E A D D E A ...,SOCIETY_SSM_W92,SOCIETY_RHIST_W92,POLPARTY,Republican,sequence,Pew_American_Trends_Panel_disagreement_100


In [9]:
# Function to dynamically gather data and create preference pairs
def create_preference_pairs(sampled_qIDs, demographic_group, output_type):
    """
    Dynamically create preference pairs by gathering data for all demographic groups.
    Args:
        sampled_qIDs (list): List of qIDs to gather data for.
        demographic_group (str): The demographic group key (e.g., "POLPARTY").
        output_type (str): Output type for the task (e.g., "sequence").
    Returns:
        pd.DataFrame: A DataFrame containing preference pairs.
    """
    # Dynamically identify unique demographics using get_few_shot_training_examples
    unique_demographics = {"Republican", "Democrat"}

    # for qID in sampled_qIDs:
    #     # Extract available demographics (simulated here as a function to identify groups)
    #     demographics = get_available_demographics(
    #         qID,
    #         wave="Pew_American_Trends_Panel_disagreement_100",
    #         demographic_group=demographic_group,
    #         dataset="opinionqa",
    #     )
    #     unique_demographics.update(demographics)

    # Validate that at least two demographics are available
    if len(unique_demographics) < 2:
        raise ValueError("At least two demographic groups are required to create preference pairs.")

    # Gather data for each demographic group
    group_datasets = {}
    for demographic in unique_demographics:
        group_datasets[demographic] = []
        for qID in sampled_qIDs:
            group_data = get_few_shot_training_examples(
                qID,
                wave="Pew_American_Trends_Panel_disagreement_100",
                demographic_group=demographic_group,
                demographic=demographic,
                output_type=output_type,
                dataset="opinionqa",
                n_shots=5,
                n_simulations_per_shot=1,
            )
            group_datasets[demographic].append(group_data)

        # Combine all data for this demographic group
        group_datasets[demographic] = pd.concat(group_datasets[demographic])

    # Generate preference pairs
    pairs = []
    for preferred_demo, preferred_data in group_datasets.items():
        for less_preferred_demo, less_preferred_data in group_datasets.items():
            if preferred_demo != less_preferred_demo:
                for _, preferred_row in preferred_data.iterrows():
                    for _, less_preferred_row in less_preferred_data.iterrows():
                        pairs.append({
                            "preferred_input": preferred_row["input"],
                            "preferred_output": preferred_row["output"],
                            "less_preferred_input": less_preferred_row["input"],
                            "less_preferred_output": less_preferred_row["output"],
                        })

    return pd.DataFrame(pairs)

# Example usage
preference_dataset = create_preference_pairs(
    sampled_qIDs=sampled_qIDs,
    demographic_group="POLPARTY",
    output_type="sequence"
)

# Preview the preference dataset
preference_dataset.head(3)


/content/gdrive/MyDrive/Benchmarking-Distributional-Alignment-of-Small-Language-Models/utils.py:138: RuntimeWarning: invalid value encountered in divide
  icl_values = np.array(icl_values)/np.sum(icl_values)


,preferred_input,preferred_output,less_preferred_input,less_preferred_output
0,Please simulate 30 samples from a group of Rep...,Answer: D D C C C B D B D D D D A B D A B D B ...,Please simulate 30 samples from a group of Dem...,Answer: E E C E C E B E E C E B E C E E B D E ...
1,Please simulate 30 samples from a group of Rep...,Answer: D D C C C B D B D D D D A B D A B D B ...,Please simulate 30 samples from a group of Dem...,Answer: B D C D D E B B D D D B C D B B D B B ...
2,Please simulate 30 samples from a group of Rep...,Answer: D D C C C B D B D D D D A B D A B D B ...,Please simulate 30 samples from a group of Dem...,Answer: F F E D F F D F F D C D E D D D A F A ...


In [10]:
from datasets import Dataset
def prepare_dpo_dataset(preference_dataset):
    dpo_format = {
        "prompt": [],
        "chosen": [],
        "rejected": []
    }

    for _, row in preference_dataset.iterrows():
        # Since the input (prompt) should be the same for both preferred and less preferred
        # We'll use the preferred_input
        dpo_format["prompt"].append(row["preferred_input"])
        dpo_format["chosen"].append(row["preferred_output"])
        dpo_format["rejected"].append(row["less_preferred_output"])

    return Dataset.from_dict(dpo_format)
dpo_dataset = prepare_dpo_dataset(preference_dataset)

In [11]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print("CUDA is available.")

    # Reset peak stats
    torch.cuda.reset_peak_memory_stats()

    # Print memory usage
    print(f"Allocated memory: {torch.cuda.memory_allocated(device) / (1024 ** 2):.2f} MiB")
    print(f"Reserved memory: {torch.cuda.memory_reserved(device) / (1024 ** 2):.2f} MiB")

    # Memory summary (requires PyTorch 1.5.0 or later)
    try:
        print(torch.cuda.memory_summary(device=device, abbreviated=True))
    except KeyError as e:
        print("Error in memory summary:", e)
else:
    print("CUDA is not available.")

CUDA is available.
Allocated memory: 10050.94 MiB
Reserved memory: 10104.00 MiB
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  10050 MiB |  10050 MiB |  10050 MiB |    512 B   |
|---------------------------------------------------------------------------|
| Active memory         |  10050 MiB |  10050 MiB |  10050 MiB |    512 B   |
|---------------------------------------------------------------------------|
| Requested memory      |   9972 MiB |   9972 MiB |   9972 MiB

In [12]:
print(torch.cuda.is_available())

True


# Fine-tune and prompting the fine-tuned model

In [14]:
from trl import DPOTrainer
import torch
from transformers import TrainingArguments
from peft import LoraConfig

lora_config = LoraConfig(
    r=4,
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "o_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


AttributeError: 'TrainingArguments' object has no attribute 'model_init_kwargs'

In [18]:
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch.cuda.empty_cache()
training_args = DPOConfig(
    output_dir="checkpoint",
    logging_steps=10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    max_steps=100,
    learning_rate=1e-4)
trainer = DPOTrainer(model=model, args=training_args, processing_class=tokenizer, train_dataset=dpo_dataset, peft_config=lora_config)

Extracting prompt from train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [19]:
trainer.model.print_trainable_parameters()

trainable params: 5,191,680 || all params: 2,619,533,568 || trainable%: 0.1982


In [20]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
10,0.691400
20,0.654600
30,0.606100
40,0.516900
50,0.405300
60,0.354000
70,0.373500
80,0.264300
90,0.204100
100,0.238400


TrainOutput(global_step=100, training_loss=0.43085805177688596, metrics={'train_runtime': 215.6408, 'train_samples_per_second': 3.71, 'train_steps_per_second': 0.464, 'total_flos': 0.0, 'train_loss': 0.43085805177688596, 'epoch': 0.16})

In [21]:
def apply_chat_template(row):
    messages = [{"role": "user", "content": row["input"]}]
    nobos = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True)[1:]
    return tokenizer.decode(nobos)

test_pool = get_test_questions_with_distributions(
    seen_qIDs={},
    demographic_group=demographic_group,
    demographic=demographic,
)
test_qIDs = json.load(open(n_testing_qIDs))

In [ ]:
k = 1
success_rates = []
probabilities_list = []
for test_qID in test_qIDs:
    print("Evaluating:", test_qID)
    # test_qID = "ECON5_d_W54"
    n = (sum(test_pool[test_qID][demographic].values()))
    MC_options = list(test_pool[test_qID][demographic].keys())
    all_options, probs = [], []
    for i, option in enumerate(MC_options):
        all_options.append(options[i])
        probs.append(test_pool[test_qID][demographic][option]/n)
    golden_dist = dict(zip(all_options, probs))
    instruction = get_zeroshot_prompt_opinionqa(test_qID, output_type="sequence")

    instruction = apply_chat_template({"input": instruction})
    model_inputs = tokenizer(instruction, return_tensors="pt").to(device)
    successful_parsings = 0
    total_attempts = 0
    while successful_parsings < k:
      outputs = model.generate(
        **model_inputs,
        max_new_tokens=36,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        early_stopping=True
      )
      response = tokenizer.decode(outputs[0][model_inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
      print(response)
      success, result = parse_answers(response, all_options, answer_tag=False)
      total_attempts += 1
      if success:
          successful_parsings += 1
          probabilities_list.append([golden_dist, result["probabilities"]])
      success_rate = successful_parsings / total_attempts
      success_rates += [success_rate]
success_rate = np.array(success_rates).mean()
print("Success rate:", success_rate)

Evaluating: WHYNOTBIZF2G_W36
A. Major reason.
B. Minor reason. 
C. Not a reason. 
D. Refused. 

A. Major reason. 
B. Minor reason. 
C. Not a reason.
D. Refused. 
... 
D. Refused.
A. Major reason. B. Minor reason. C. Not a reason. D. Refused.  

I can't provide 30 "answers" to a
B. Minor reason. 

A. Major reason. 
B. Major reason. 
C. Not a reason. 
D. Refused. 
A. Major reason. 

A. Major reason. 
B. Minor reason.
C. Not a reason.
D. Refused. 

It is important to remember that discrimination has no
A. Major reason. 
B. Minor reason.
C. Not a reason. 
D. Refused.
 
It's important to provide relevant
A. Major reason. 
B. Major reason. 
C. Not a reason.
D. Refused.
A. Major reason.
B. Major
A. Major reason. B. Minor reason. C. Not a reason. D. Refused. 


The prompt asked to simulate samples from a group of Democrats.  
A. Major reason. B. Minor reason. C. Not a reason. D. Refused. 
 
It is not accurate to say women are held to higher


In [ ]:
distances = compute_l1_values(probabilities_list)
np.mean(distances)

0.9722800183566654

In [ ]:
json.dump(distances, open("distance_lora_dpo.json", "w"))

In [ ]:
instruction = "Tell me about US politics."

instruction = apply_chat_template({"input": instruction})
model_inputs = tokenizer(instruction, return_tensors="pt").to(device)

outputs = model.generate(
    **model_inputs, max_new_tokens=128, do_sample=True,
    eos_token_id=tokenizer.eos_token_id, early_stopping=True
)
response = tokenizer.decode(outputs[0][model_inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


## US Politics: A Complex Tapestry of Ideologies and Systems

The US political system is a fascinating and intricate one.  Here's a breakdown of key elements:

**1. The Basic Structure:**
- **Federal System with Strong State Power:** Power is divided between a national (federal) government and individual states.  
- **National Government Powers (Enumerated):**  A specific power set is outlined in the Constitution for the national government, including:
    - Declaring war
    - Establishing a legal code
    - Collecting taxes
- **States and their Powers:** States maintain most of the power,
